<a href="https://colab.research.google.com/github/LC1332/Mini-Luotuo/blob/main/3.5B_minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIQA助手

原项目地址https://github.com/LC1332/Mini-Luotuo

包含3.5B模型的inference代码

安装必要的环境

In [ ]:
%pip install bitsandbytes datasets sentencepiece transformers accelerate

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

def generate_prompt(instruction, input=None, prior_knowledge=""):
    prior_knowledge = f"### 先验知识:\n{prior_knowledge}\n"

    if input:
        return f"{prior_knowledge}Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        return f"{prior_knowledge}Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n"


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('Midkey/GPT2-3.5B-chinese-ft-luotuo')

model = GPT2LMHeadModel.from_pretrained('Midkey/GPT2-3.5B-chinese-ft-luotuo')

In [ ]:
def evaluate(
    instruction,
    input="想要编辑规则，就点击页面右侧导航栏的规则编辑器",
    max_new_tokens=256,
):
    prompt = generate_prompt(instruction, input)
    
    inputs = tokenizer(prompt, return_tensors="pt")
    # input_ids = inputs["input_ids"].cuda()
    input_ids = inputs["input_ids"]
    eos_token_id = 50256
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
            pad_token_id=eos_token_id
        )
    for s in generation_output.sequences:
        decode_s = s[len(input_ids[0]):]
        if decode_s[-1] == eos_token_id:
            decode_s = decode_s[:-1]
        output = tokenizer.decode(decode_s)
        print("Response:", output)

In [ ]:
instruction = '你好'
evaluate(instruction)

In [ ]:
instruction = '我该在哪里编辑规则?'
evaluate(instruction)

In [ ]:
while 1:
    instruction = input('Ask something: ')
    evaluate(instruction)